In [1]:
import sys
import os
sys.path.append('../')
sys.path.append('../WD_models')

### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from astropy.io import fits
from astropy.table import Table, Column, MaskedColumn, join, unique
from astropy.table import *

from tqdm import tqdm
import pandas as pd
#import WD_models

In [2]:
#from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import table
import astropy.units as u

wd_c1 = Table.read('sdss5/spAll-v6_1_0.fits')


In [3]:
wd_c1[-2]

PROGRAMNAME,SURVEY,FIELDQUALITY,FIELDSN2,EXP_DISP_MED,FIBERID_LIST,LAMBDA_EFF,BLUEFIBER,ZOFFSET,XFOCAL,YFOCAL,CALIBFLUX,CALIBFLUX_IVAR,GAIA_BP,GAIA_RP,GAIA_G,CADENCE,FIRSTCARTON,CARTON_TO_TARGET_PK,RACAT,DECCAT,COORD_EPOCH,PMRA,PMDEC,PARALLAX,CATALOGID,CATALOGID_V0,CATALOGID_V0P5,GAIA_ID_DR2,FIBER2MAG,PSFMAG,CATDB_MAG,OPTICAL_PROV,OBS,FIELD,DESIGNS,CONFIGS,NEXP,EXPTIME,AIRMASS,SEEING20,SEEING50,SEEING80,ASSIGNED,ON_TARGET,VALID,HEALPIX,HEALPIXGRP,HEALPIX_PATH,MJD_FINAL,MJD_LIST,TAI_LIST,FIELDSNR2G_LIST,FIELDSNR2R_LIST,FIELDSNR2I_LIST,RA_LIST,DEC_LIST,DELTA_RA_LIST,DELTA_DEC_LIST,MOON_DIST,MOON_PHASE,EBV,EBV_TYPE,WISE_MAG,TWOMASS_MAG,GUVCAT_MAG,FIBER_OFFSET,SPEC_FILE,XCSAO_RV,XCSAO_ERV,XCSAO_RXC,XCSAO_TEFF,XCSAO_ETEFF,XCSAO_LOGG,XCSAO_ELOGG,XCSAO_FEH,XCSAO_EFEH,SDSSV_BOSS_TARGET0,MJD,TARGET_INDEX,RUN2D,RUN1D,OBJTYPE,FIBER_RA,FIBER_DEC,PLUG_RA,PLUG_DEC,CLASS,SUBCLASS,Z,Z_ERR,RCHI2,DOF,RCHI2DIFF,TFILE,TCOLUMN,NPOLY,THETA,VDISP,VDISP_ERR,VDISPZ,VDISPZ_ERR,VDISPCHI2,VDISPNPIX,VDISPDOF,WAVEMIN,WAVEMAX,WCOVERAGE,ZWARNING,SN_MEDIAN,SN_MEDIAN_ALL,CHI68P,FRACNSIGMA,FRACNSIGHI,FRACNSIGLO,SPECTROFLUX,SPECTROFLUX_IVAR,SPECTROSYNFLUX,SPECTROSYNFLUX_IVAR,SPECTROSKYFLUX,ANYANDMASK,ANYORMASK,SPEC1_G,SPEC1_R,SPEC1_I,SPEC2_G,SPEC2_R,SPEC2_I,Z_NOQSO,Z_ERR_NOQSO,ZNUM_NOQSO,ZWARNING_NOQSO,CLASS_NOQSO,SUBCLASS_NOQSO,RCHI2DIFF_NOQSO,SPECOBJID
bytes13,bytes13,bytes4,float32,float64,bytes111,float32,int32,float32,bytes251,bytes251,float32[5],float32[5],float32,float32,float32,bytes13,bytes48,bytes22,float64,float64,float32,float32,float32,float32,int64,int64,int64,int64,float32[5],float32[5],float32[5],bytes26,bytes3,int32,bytes125,bytes167,int16,int16,float32,float32,float32,float32,bytes55,bytes55,bytes55,int32,int16,bytes76,float64,bytes167,bytes307,bytes139,bytes147,bytes152,bytes307,bytes279,bytes111,bytes111,bytes139,bytes139,float32,bytes11,float32[4],float32[3],float32[2],float32,bytes42,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,int32,int32,bytes6,bytes6,bytes16,float64,float64,float64,float64,bytes6,bytes21,float32,float32,float32,int32,float32,bytes24,int32[10],int32,float32[10],float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,int32,float32[5],float32,float32,float32[10],float32[10],float32[10],float32[5],float32[5],float32[5],float32[5],float32[5],int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,bytes6,bytes21,float32,int64
,,good,12.7535515,-1.0,217 217 217 217,--,0,0.0,146.460 146.460 146.460 146.460,-18.021 -18.021 -18.021 -18.021,0.0 .. 0.0,0.0 .. 0.0,--,--,--,,,-999,--,--,--,--,--,--,0,0,0,-1,-- .. --,-- .. --,-- .. --,,APO,112361,109813 109813 109814 109814,8066 8066 8067 8067,4,3600,1.3337468,1.198143,1.2871485,1.474061,0 0 0 0,0 0 0 0,1 1 1 1,0,0,,59940.08030879989,59940 59940 59940 59940,5178821386 5178822370 5178823429 5178824412,1.22 1.66 1.36 1.46,5.01 6.22 4.96 5.52,3.96 5.16 4.13 4.79,36.444091 36.444091 36.444091 36.444091,-5.097355 -5.097355 -5.097355 -5.097355,0.0 0.0 0.0 0.0,0.0 0.0 0.0 0.0,65.7 65.6 65.4 65.3,0.19 0.19 0.19 0.19,0.02895467,SFD,-- .. --,-- .. --,-- .. --,0.0,spec-112361-59940-u022546.6-050550.5.fits,193.11948,17.499353,1.7182446,4809.273,1307.1389,2.9295957,1.4529663,-0.8741909,1.0371716,0,59940,508,v6_1_0,v6_1_0,,36.444091,-5.0973553,0.0,0.0,QSO,,1.3844838,0.002286237,0.7959811,4539,0.0007853508,spEigenQSO-58596.fits,0 .. 9,0,-7.480469e-06 .. 0.00059424044,0.0,0.0,0.0,0.0,0.0,0.0,0,3601.6372,10327.613,0.4549,132,0.86009896 .. 0.57791674,0.60251874,0.87159383,0.25148383 .. 0.0,0.13343592 .. 0.0,0.11804792 .. 0.0,-1.191027 .. 0.5548161,0.35832885 .. 1.1126714,-1.0531044 .. 0.2856037,1.8749425 .. 1.9623202,17.942547 .. 132.30573,83886081,256835585,12.7535515,48.22551,41.64344,0.0,0.0,0.0,0.5927959,0.00024895364,6,132,GALAXY,,8.773804e-05,-2619829278470189056


In [4]:
wd_c1['SPEC_FILE']

spec-015000-59146-4375786564.fits
spec-015000-59146-4375786575.fits
spec-015000-59146-4375787016.fits
spec-015000-59146-4375787024.fits
spec-015000-59146-4375787075.fits
spec-015000-59146-4375787211.fits
spec-015000-59146-4375787390.fits
spec-015000-59146-4375787467.fits
spec-015000-59146-4375787547.fits
spec-015000-59146-4375787802.fits
spec-015000-59146-4375787870.fits


In [5]:
def strip(catalog):  
    drop_cols = []
    keep_cols = ['CATALOGID', 'FIBER_RA', 'FIBER_DEC', 'MJD', 'FIELD', 'FIELDSN2', 'SPEC_FILE']
        
    for key in catalog.keys():
        if key not in keep_cols:
            drop_cols.append(key)
                 
    catalog.remove_columns(drop_cols)
    
    return catalog

wd_c1 = strip(wd_c1)
ms_c1 = wd_c1.copy()

In [6]:
print(wd_c1.keys())

['FIELDSN2', 'CATALOGID', 'FIELD', 'SPEC_FILE', 'MJD', 'FIBER_RA', 'FIBER_DEC']


In [7]:
print(ms_c1.keys())

['FIELDSN2', 'CATALOGID', 'FIELD', 'SPEC_FILE', 'MJD', 'FIBER_RA', 'FIBER_DEC']


In [8]:
catalog = Table.read('data/elbadry.fits.gz')

catalog = catalog[catalog['binary_type'] == b'WDMS']

In [9]:
def wd_sep(bp_rp):
    return 3.25*bp_rp + 9.625

wd_obj = []
ms_obj = []
drop = []

for i in tqdm (range(len(catalog))):
    mg1 = wd_sep(catalog['bp_rp1'][i])
    mg2 = wd_sep(catalog['bp_rp2'][i])
    
    M1 = catalog['phot_g_mean_mag1'][i] + 5 * (np.log10(catalog['parallax1'][i] / 100))
    M2 = catalog['phot_g_mean_mag2'][i] + 5 * (np.log10(catalog['parallax2'][i] / 100))
    
    if M1 > mg1 and M2 < mg2:
        wd_obj.append(1)
        ms_obj.append(2)
    elif M2 > mg2 and M1 < mg1:
        wd_obj.append(2)
        ms_obj.append(1)
    else:
        drop.append(i)
               
catalog.remove_rows(drop)
catalog['wd_obj'] = wd_obj
catalog['ms_obj'] = ms_obj

100%|██████████| 22563/22563 [00:00<00:00, 28589.75it/s]


In [10]:
def separate(catalog, column, ms_obj, wd_obj, newname = ''):      
    mstemp_arr = [ catalog[str(column) + str(ms_obj[i])][i] for i in range(len(ms_obj)) ]
    wdtemp_arr = [ catalog[str(column) + str(wd_obj[i])][i] for i in range(len(wd_obj)) ]
    
    catalog['ms_' + str(column)] = mstemp_arr
    catalog['wd_' + str(column)] = wdtemp_arr
    return catalog

### ---

convert_cols = ['source_id', 'parallax', 'parallax_over_error', 'phot_g_mean_mag', 'phot_g_mean_flux',
                'phot_g_mean_flux_error', 'phot_bp_mean_mag', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error',
                'phot_rp_mean_mag', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'bp_rp', 'ra', 'ra_error', 'dec', 'dec_error', 'l', 'b']

for col in convert_cols:
    catalog = separate(catalog, col, ms_obj, wd_obj)
    
catalog['wd_m_g'] = catalog['wd_phot_g_mean_mag'] + 5 * np.log10(catalog['wd_parallax'] / 100)
catalog['ms_m_g'] = catalog['ms_phot_g_mean_mag'] + 5 * np.log10(catalog['ms_parallax'] / 100)

In [11]:
from astropy.table import unique, Table

wd_c1['wd_pos'] = SkyCoord(wd_c1['FIBER_RA'], wd_c1['FIBER_DEC'], unit='deg')
ms_c1['ms_pos'] = SkyCoord(ms_c1['FIBER_RA'], ms_c1['FIBER_DEC'], unit='deg')

catalog['wd_pos'] = SkyCoord(catalog['wd_ra'], catalog['wd_dec'], unit='deg')
catalog['ms_pos'] = SkyCoord(catalog['ms_ra'], catalog['ms_dec'], unit='deg')

In [12]:
wd_c1[['wd_pos']]

wd_pos
"deg,deg"
SkyCoord
"35.103846,-6.2921282"
"35.01667,-6.2686349"
"34.90911,-6.0668356"
"34.972713,-6.1898776"
"34.991105,-6.0930068"
"35.056275,-6.0171515"
"35.407754,-6.4484517"
"35.198357,-6.2015798"


In [13]:
from inspect import getmembers, isfunction
from astropy import table

print(getmembers(table, isfunction))

[('dstack', <function dstack at 0x15553331b940>), ('hstack', <function hstack at 0x15553331ba60>), ('join', <function join at 0x15553331b820>), ('join_distance', <function join_distance at 0x15553331b790>), ('join_skycoord', <function join_skycoord at 0x15553331b700>), ('represent_mixins_as_columns', <function represent_mixins_as_columns at 0x15553331d310>), ('setdiff', <function setdiff at 0x15553331b8b0>), ('unique', <function unique at 0x15553331baf0>), ('vstack', <function vstack at 0x15553331b9d0>)]


In [14]:
import astropy

print(astropy.__version__)

5.2.2


In [15]:
from astropy.table import join_skycoord

# Join WDs in SDSS-5
wd_catalog = unique(join(catalog, wd_c1, join_funcs={'wd_pos': join_skycoord(1 * u.arcsecond)}), keys = 'wd_pos_id')
for key in wd_c1.keys():
    if key != 'wd_pos':
        wd_catalog.rename_column(key, 'wd_' + key)

# Join MS in SDSS-5
ms_catalog = unique(join(catalog, ms_c1, join_funcs={'ms_pos': join_skycoord(1 * u.arcsecond)}), keys = 'ms_pos_id')
for key in ms_c1.keys():
    if key != 'ms_pos':
        ms_catalog.rename_column(key, 'ms_' + key)

for key in catalog.keys():
    if key != 'ms_source_id':
        try:
            ms_catalog.remove_column(key)
        except:
            ms_catalog.remove_column(key + '_1')
            ms_catalog.rename_column(key + '_2', key)
ms_sdss5_wd_sdss5 = join(wd_catalog, ms_catalog, keys = 'ms_source_id')

In [16]:
ms_sdss5_wd_sdss5

wd_pos_id,solution_id1,solution_id2,source_id1,source_id2,random_index1,random_index2,ref_epoch1,ref_epoch2,ra1,ra2,ra_error1,ra_error2,dec1,dec2,dec_error1,dec_error2,parallax1,parallax2,parallax_error1,parallax_error2,parallax_over_error1,parallax_over_error2,pm1,pm2,pmra1,pmra2,pmra_error1,pmra_error2,pmdec1,pmdec2,pmdec_error1,pmdec_error2,ra_dec_corr1,ra_dec_corr2,ra_parallax_corr1,ra_parallax_corr2,ra_pmra_corr1,ra_pmra_corr2,ra_pmdec_corr1,ra_pmdec_corr2,dec_parallax_corr1,dec_parallax_corr2,dec_pmra_corr1,dec_pmra_corr2,dec_pmdec_corr1,dec_pmdec_corr2,parallax_pmra_corr1,parallax_pmra_corr2,parallax_pmdec_corr1,parallax_pmdec_corr2,pmra_pmdec_corr1,pmra_pmdec_corr2,astrometric_n_obs_al1,astrometric_n_obs_al2,astrometric_n_obs_ac1,astrometric_n_obs_ac2,astrometric_n_good_obs_al1,astrometric_n_good_obs_al2,astrometric_n_bad_obs_al1,astrometric_n_bad_obs_al2,astrometric_gof_al1,astrometric_gof_al2,astrometric_chi2_al1,astrometric_chi2_al2,astrometric_excess_noise1,astrometric_excess_noise2,astrometric_excess_noise_sig1,astrometric_excess_noise_sig2,astrometric_params_solved1,astrometric_params_solved2,astrometric_primary_flag1,astrometric_primary_flag2,nu_eff_used_in_astrometry1,nu_eff_used_in_astrometry2,pseudocolour1,pseudocolour2,pseudocolour_error1,pseudocolour_error2,ra_pseudocolour_corr1,ra_pseudocolour_corr2,dec_pseudocolour_corr1,dec_pseudocolour_corr2,parallax_pseudocolour_corr1,parallax_pseudocolour_corr2,pmra_pseudocolour_corr1,pmra_pseudocolour_corr2,pmdec_pseudocolour_corr1,pmdec_pseudocolour_corr2,astrometric_matched_transits1,astrometric_matched_transits2,visibility_periods_used1,visibility_periods_used2,astrometric_sigma5d_max1,astrometric_sigma5d_max2,matched_transits1,matched_transits2,new_matched_transits1,new_matched_transits2,matched_transits_removed1,matched_transits_removed2,ipd_gof_harmonic_amplitude1,ipd_gof_harmonic_amplitude2,ipd_gof_harmonic_phase1,ipd_gof_harmonic_phase2,ipd_frac_multi_peak1,ipd_frac_multi_peak2,ipd_frac_odd_win1,ipd_frac_odd_win2,ruwe1,ruwe2,scan_direction_strength_k11,scan_direction_strength_k12,scan_direction_strength_k21,scan_direction_strength_k22,scan_direction_strength_k31,scan_direction_strength_k32,scan_direction_strength_k41,scan_direction_strength_k42,scan_direction_mean_k11,scan_direction_mean_k12,scan_direction_mean_k21,scan_direction_mean_k22,scan_direction_mean_k31,scan_direction_mean_k32,scan_direction_mean_k41,scan_direction_mean_k42,duplicated_source1,duplicated_source2,phot_g_n_obs1,phot_g_n_obs2,phot_g_mean_flux1,phot_g_mean_flux2,phot_g_mean_flux_error1,phot_g_mean_flux_error2,phot_g_mean_flux_over_error1,phot_g_mean_flux_over_error2,phot_g_mean_mag1,phot_g_mean_mag2,phot_bp_n_obs1,phot_bp_n_obs2,phot_bp_mean_flux1,phot_bp_mean_flux2,phot_bp_mean_flux_error1,phot_bp_mean_flux_error2,phot_bp_mean_flux_over_error1,phot_bp_mean_flux_over_error2,phot_bp_mean_mag1,phot_bp_mean_mag2,phot_rp_n_obs1,phot_rp_n_obs2,phot_rp_mean_flux1,phot_rp_mean_flux2,phot_rp_mean_flux_error1,phot_rp_mean_flux_error2,phot_rp_mean_flux_over_error1,phot_rp_mean_flux_over_error2,phot_rp_mean_mag1,phot_rp_mean_mag2,phot_bp_n_contaminated_transits1,phot_bp_n_contaminated_transits2,phot_bp_n_blended_transits1,phot_bp_n_blended_transits2,phot_rp_n_contaminated_transits1,phot_rp_n_contaminated_transits2,phot_rp_n_blended_transits1,phot_rp_n_blended_transits2,phot_proc_mode1,phot_proc_mode2,phot_bp_rp_excess_factor1,phot_bp_rp_excess_factor2,bp_rp1,bp_rp2,bp_g1,bp_g2,g_rp1,g_rp2,dr2_radial_velocity1,dr2_radial_velocity2,dr2_radial_velocity_error1,dr2_radial_velocity_error2,dr2_rv_nb_transits1,dr2_rv_nb_transits2,dr2_rv_template_teff1,dr2_rv_template_teff2,dr2_rv_template_logg1,dr2_rv_template_logg2,dr2_rv_template_fe_h1,dr2_rv_template_fe_h2,l1,l2,b1,b2,ecl_lon1,ecl_lon2,ecl_lat1,ecl_lat2,pairdistance,sep_AU,binary_type,Sigma18,R_chance_align,dr2_source_id1,dr2_source_id2,dr2_parallax1,dr2_parallax2,dr2_parallax_error1,dr2_parallax_error2,dr2_pmra1,dr2_pmra2,dr2_pmdec1,dr2_pmdec2,dr2_pmra_

In [17]:
from astropy.table import Table, vstack, hstack

from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c

# Fetch targets with MS RVs in Gaia

ADQL_CODE1 = """SELECT sdss.original_ext_source_id as ms_bestobjid, gaia.source_id as ms_source_id, gaia.radial_velocity, gaia.radial_velocity_error
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss
    ON gaia.source_id = sdss.source_id      
    WHERE gaia.source_id in {} AND
    radial_velocity IS NOT null AND
    radial_velocity_error < 2""".format(tuple(wd_catalog['ms_source_id']))
d1 = (Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results())
ms_gaia_wd_sdss5 = join(wd_catalog, d1, keys = 'ms_source_id')
    
# Fetch targets with MS spectra in SDSS-4

ADQL_CODE1 = """SELECT sdss.original_ext_source_id as ms_bestobjid, gaia.source_id as ms_source_id, gaia.radial_velocity, gaia.radial_velocity_error
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss
    ON gaia.source_id = sdss.source_id      
    WHERE gaia.source_id in {}""".format(tuple(wd_catalog['ms_source_id']))
gd2 = Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results() 

stardats = []
iters = len(gd2) // 100

for i in tqdm(range(iters)):
    SDSS_QUERY = """select sp.bestObjID, sp.plate, sp.mjd, sp.fiberID
        from dbo.SpecObjAll as sp
        where sp.bestObjID in {}""".format(tuple(gd2['ms_bestobjid'][100*i:100*i+100]))
    
    f = SDSS.query_sql(SDSS_QUERY)
    if f is not None:
        stardats.append(SDSS.query_sql(SDSS_QUERY))

d2 = vstack(stardats)
d2.rename_column('bestObjID', 'bestobjid')
d2.rename_column('fiberID', 'fiberid')

print(d2.keys())

for key in d2.keys():
    d2.rename_column(key, 'ms_' + key)

d2 = join(d2, gd2, keys = 'ms_bestobjid')
ms_sdss4_wd_sdss5 = join(wd_catalog, d2, keys = 'ms_source_id')
#except:
#    print('No objects with MS spectra in SDSS-4 and WD spectra in SDSS-5')
#    ms_sdss4_wd_sdss5 = [0]
#ms_in_sdss4 = unique(d2, keys = 'wd_bestobjid')

100%|██████████| 6/6 [00:00<00:00, 219.64it/s]

['bestobjid', 'plate', 'mjd', 'fiberid']


In [18]:
ms_sdss5_wd_sdss4 = Table.read('sdss5/temp_ms_sdss5_wd_sdss4.fits')
ms_sdss5_wd_sdss4['ms_pos'] = SkyCoord(ms_sdss5_wd_sdss4['ms_ra'], ms_sdss5_wd_sdss4['ms_dec'], unit='deg')

ms_sdss5_wd_sdss4 = unique(join(ms_sdss5_wd_sdss4, ms_c1, join_funcs={'ms_pos': join_skycoord(1 * u.arcsecond)}), keys = 'ms_pos_id')
for key in ms_c1.keys():
    if key != 'ms_pos':
        ms_sdss5_wd_sdss4.rename_column(key, 'ms_' + key)

In [19]:
ms_gaia_wd_sdss5

wd_pos_id,solution_id1,solution_id2,source_id1,source_id2,random_index1,random_index2,ref_epoch1,ref_epoch2,ra1,ra2,ra_error1,ra_error2,dec1,dec2,dec_error1,dec_error2,parallax1,parallax2,parallax_error1,parallax_error2,parallax_over_error1,parallax_over_error2,pm1,pm2,pmra1,pmra2,pmra_error1,pmra_error2,pmdec1,pmdec2,pmdec_error1,pmdec_error2,ra_dec_corr1,ra_dec_corr2,ra_parallax_corr1,ra_parallax_corr2,ra_pmra_corr1,ra_pmra_corr2,ra_pmdec_corr1,ra_pmdec_corr2,dec_parallax_corr1,dec_parallax_corr2,dec_pmra_corr1,dec_pmra_corr2,dec_pmdec_corr1,dec_pmdec_corr2,parallax_pmra_corr1,parallax_pmra_corr2,parallax_pmdec_corr1,parallax_pmdec_corr2,pmra_pmdec_corr1,pmra_pmdec_corr2,astrometric_n_obs_al1,astrometric_n_obs_al2,astrometric_n_obs_ac1,astrometric_n_obs_ac2,astrometric_n_good_obs_al1,astrometric_n_good_obs_al2,astrometric_n_bad_obs_al1,astrometric_n_bad_obs_al2,astrometric_gof_al1,astrometric_gof_al2,astrometric_chi2_al1,astrometric_chi2_al2,astrometric_excess_noise1,astrometric_excess_noise2,astrometric_excess_noise_sig1,astrometric_excess_noise_sig2,astrometric_params_solved1,astrometric_params_solved2,astrometric_primary_flag1,astrometric_primary_flag2,nu_eff_used_in_astrometry1,nu_eff_used_in_astrometry2,pseudocolour1,pseudocolour2,pseudocolour_error1,pseudocolour_error2,ra_pseudocolour_corr1,ra_pseudocolour_corr2,dec_pseudocolour_corr1,dec_pseudocolour_corr2,parallax_pseudocolour_corr1,parallax_pseudocolour_corr2,pmra_pseudocolour_corr1,pmra_pseudocolour_corr2,pmdec_pseudocolour_corr1,pmdec_pseudocolour_corr2,astrometric_matched_transits1,astrometric_matched_transits2,visibility_periods_used1,visibility_periods_used2,astrometric_sigma5d_max1,astrometric_sigma5d_max2,matched_transits1,matched_transits2,new_matched_transits1,new_matched_transits2,matched_transits_removed1,matched_transits_removed2,ipd_gof_harmonic_amplitude1,ipd_gof_harmonic_amplitude2,ipd_gof_harmonic_phase1,ipd_gof_harmonic_phase2,ipd_frac_multi_peak1,ipd_frac_multi_peak2,ipd_frac_odd_win1,ipd_frac_odd_win2,ruwe1,ruwe2,scan_direction_strength_k11,scan_direction_strength_k12,scan_direction_strength_k21,scan_direction_strength_k22,scan_direction_strength_k31,scan_direction_strength_k32,scan_direction_strength_k41,scan_direction_strength_k42,scan_direction_mean_k11,scan_direction_mean_k12,scan_direction_mean_k21,scan_direction_mean_k22,scan_direction_mean_k31,scan_direction_mean_k32,scan_direction_mean_k41,scan_direction_mean_k42,duplicated_source1,duplicated_source2,phot_g_n_obs1,phot_g_n_obs2,phot_g_mean_flux1,phot_g_mean_flux2,phot_g_mean_flux_error1,phot_g_mean_flux_error2,phot_g_mean_flux_over_error1,phot_g_mean_flux_over_error2,phot_g_mean_mag1,phot_g_mean_mag2,phot_bp_n_obs1,phot_bp_n_obs2,phot_bp_mean_flux1,phot_bp_mean_flux2,phot_bp_mean_flux_error1,phot_bp_mean_flux_error2,phot_bp_mean_flux_over_error1,phot_bp_mean_flux_over_error2,phot_bp_mean_mag1,phot_bp_mean_mag2,phot_rp_n_obs1,phot_rp_n_obs2,phot_rp_mean_flux1,phot_rp_mean_flux2,phot_rp_mean_flux_error1,phot_rp_mean_flux_error2,phot_rp_mean_flux_over_error1,phot_rp_mean_flux_over_error2,phot_rp_mean_mag1,phot_rp_mean_mag2,phot_bp_n_contaminated_transits1,phot_bp_n_contaminated_transits2,phot_bp_n_blended_transits1,phot_bp_n_blended_transits2,phot_rp_n_contaminated_transits1,phot_rp_n_contaminated_transits2,phot_rp_n_blended_transits1,phot_rp_n_blended_transits2,phot_proc_mode1,phot_proc_mode2,phot_bp_rp_excess_factor1,phot_bp_rp_excess_factor2,bp_rp1,bp_rp2,bp_g1,bp_g2,g_rp1,g_rp2,dr2_radial_velocity1,dr2_radial_velocity2,dr2_radial_velocity_error1,dr2_radial_velocity_error2,dr2_rv_nb_transits1,dr2_rv_nb_transits2,dr2_rv_template_teff1,dr2_rv_template_teff2,dr2_rv_template_logg1,dr2_rv_template_logg2,dr2_rv_template_fe_h1,dr2_rv_template_fe_h2,l1,l2,b1,b2,ecl_lon1,ecl_lon2,ecl_lat1,ecl_lat2,pairdistance,sep_AU,binary_type,Sigma18,R_chance_align,dr2_source_id1,dr2_source_id2,dr2_parallax1,dr2_parallax2,dr2_parallax_error1,dr2_parallax_error2,dr2_pmra1,dr2_pmra2,dr2_pmdec1,dr2_pmdec2,dr2_pmra_

In [20]:
# MS Gaia, WD SDSS-V

ms_gaia_wd_sdss5['wd_filepath'] =  ms_gaia_wd_sdss5['wd_SPEC_FILE']
ms_gaia_wd_sdss5['wd_plate'] = ms_gaia_wd_sdss5['wd_FIELD']
ms_gaia_wd_sdss5['wd_mjd'] = ms_gaia_wd_sdss5['wd_MJD']
ms_gaia_wd_sdss5['wd_fiberid'] = ms_gaia_wd_sdss5['wd_CATALOGID']

ms_gaia_wd_sdss5['uses_sdss5'] = np.ones(len(ms_gaia_wd_sdss5))
ms_gaia_wd_sdss5['wd_rv_from'] = ['sdss5' for i in range(len(ms_gaia_wd_sdss5))]
ms_gaia_wd_sdss5['ms_rv_from'] = ['gaia' for i in range(len(ms_gaia_wd_sdss5))]

ms_gaia_wd_sdss5['wd_rv'] = 999*np.ones(len(ms_gaia_wd_sdss5))
ms_gaia_wd_sdss5['wd_e_rv'] = 999*np.ones(len(ms_gaia_wd_sdss5))

ms_gaia_wd_sdss5 = unique(ms_gaia_wd_sdss5, keys = 'wd_source_id')

# MS SDSS-IV, WD SDSS-V

ms_sdss4_wd_sdss5['wd_filepath'] =  ms_sdss4_wd_sdss5['wd_SPEC_FILE']
ms_sdss4_wd_sdss5['wd_plate'] = ms_sdss4_wd_sdss5['wd_FIELD']
ms_sdss4_wd_sdss5['wd_mjd'] = ms_sdss4_wd_sdss5['wd_MJD']
ms_sdss4_wd_sdss5['wd_fiberid'] = ms_sdss4_wd_sdss5['wd_CATALOGID']

ms_sdss4_wd_sdss5['uses_sdss5'] = np.ones(len(ms_sdss4_wd_sdss5))
ms_sdss4_wd_sdss5['wd_rv_from'] = ['sdss5' for i in range(len(ms_sdss4_wd_sdss5))]
ms_sdss4_wd_sdss5['ms_rv_from'] = ['sdss4' for i in range(len(ms_sdss4_wd_sdss5))]

ms_sdss4_wd_sdss5['wd_rv'] = 999*np.ones(len(ms_sdss4_wd_sdss5))
ms_sdss4_wd_sdss5['wd_e_rv'] = 999*np.ones(len(ms_sdss4_wd_sdss5))

ms_sdss4_wd_sdss5 = unique(ms_sdss4_wd_sdss5, keys = 'wd_source_id')

# MS SDSS-V, WD SDSS-V

ms_sdss5_wd_sdss5['ms_filepath'] =  ms_sdss5_wd_sdss5['ms_SPEC_FILE']
ms_sdss5_wd_sdss5['ms_plate'] = ms_sdss5_wd_sdss5['ms_FIELD']
ms_sdss5_wd_sdss5['ms_mjd'] = ms_sdss5_wd_sdss5['ms_MJD']
ms_sdss5_wd_sdss5['ms_fiberid'] = ms_sdss5_wd_sdss5['wd_CATALOGID']

ms_sdss5_wd_sdss5['wd_filepath'] =  ms_sdss5_wd_sdss5['wd_SPEC_FILE']
ms_sdss5_wd_sdss5['wd_plate'] = ms_sdss5_wd_sdss5['wd_FIELD']
ms_sdss5_wd_sdss5['wd_mjd'] = ms_sdss5_wd_sdss5['wd_MJD']
ms_sdss5_wd_sdss5['wd_fiberid'] = ms_sdss5_wd_sdss5['wd_CATALOGID']

ms_sdss5_wd_sdss5['wd_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss5))
ms_sdss5_wd_sdss5['wd_e_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss5))

ms_sdss5_wd_sdss5['uses_sdss5'] = np.ones(len(ms_sdss5_wd_sdss5))
ms_sdss5_wd_sdss5['wd_rv_from'] = ['sdss5' for i in range(len(ms_sdss5_wd_sdss5))]
ms_sdss5_wd_sdss5['ms_rv_from'] = ['sdss5' for i in range(len(ms_sdss5_wd_sdss5))]

ms_sdss5_wd_sdss5 = unique(ms_sdss5_wd_sdss5, keys = 'wd_source_id')

# MS SDSS-V, WD SDSS-IV

ms_sdss5_wd_sdss4['ms_filepath'] =  ms_sdss5_wd_sdss4['ms_SPEC_FILE']
ms_sdss5_wd_sdss4['ms_plate'] = ms_sdss5_wd_sdss4['ms_FIELD']
ms_sdss5_wd_sdss4['ms_mjd'] = ms_sdss5_wd_sdss4['ms_MJD']
ms_sdss5_wd_sdss4['ms_fiberid'] = ms_sdss5_wd_sdss4['ms_CATALOGID']

ms_sdss5_wd_sdss4['wd_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss4))
ms_sdss5_wd_sdss4['wd_e_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss4))

ms_sdss5_wd_sdss4['uses_sdss5'] = np.ones(len(ms_sdss5_wd_sdss4))
ms_sdss5_wd_sdss4['wd_rv_from'] = ['sdss4' for i in range(len(ms_sdss5_wd_sdss4))]
ms_sdss5_wd_sdss4['ms_rv_from'] = ['sdss5' for i in range(len(ms_sdss5_wd_sdss4))]

ms_sdss5_wd_sdss4 = unique(ms_sdss5_wd_sdss4, keys = 'wd_source_id')

# Write Catalogs

ms_gaia_wd_sdss5.write('sdss5/ms_gaia_wd_sdss5.fits', overwrite = True)
ms_sdss4_wd_sdss5.write('sdss5/ms_sdss4_wd_sdss5.fits', overwrite = True)
ms_sdss5_wd_sdss5.write('sdss5/ms_sdss5_wd_sdss5.fits', overwrite = True)
ms_sdss5_wd_sdss4.write('sdss5/ms_sdss5_wd_sdss4.fits', overwrite = True)

In [21]:
print(len(ms_gaia_wd_sdss5))
print(len(unique(ms_gaia_wd_sdss5, keys = 'wd_source_id')))

print(len(ms_sdss4_wd_sdss5))
print(len(unique(ms_sdss4_wd_sdss5, keys = 'wd_source_id')))

print(len(ms_sdss5_wd_sdss5))
print(len(unique(ms_sdss5_wd_sdss5, keys = 'wd_source_id')))

print(len(ms_sdss5_wd_sdss4))
print(len(unique(ms_sdss5_wd_sdss4, keys = 'wd_source_id')))

123
123
5
5
88
88
71
71


In [22]:
ms_sdss4_wd_sdss5[ms_sdss4_wd_sdss5['ms_source_id'] == 875615158730751488]

wd_pos_id,solution_id1,solution_id2,source_id1,source_id2,random_index1,random_index2,ref_epoch1,ref_epoch2,ra1,ra2,ra_error1,ra_error2,dec1,dec2,dec_error1,dec_error2,parallax1,parallax2,parallax_error1,parallax_error2,parallax_over_error1,parallax_over_error2,pm1,pm2,pmra1,pmra2,pmra_error1,pmra_error2,pmdec1,pmdec2,pmdec_error1,pmdec_error2,ra_dec_corr1,ra_dec_corr2,ra_parallax_corr1,ra_parallax_corr2,ra_pmra_corr1,ra_pmra_corr2,ra_pmdec_corr1,ra_pmdec_corr2,dec_parallax_corr1,dec_parallax_corr2,dec_pmra_corr1,dec_pmra_corr2,dec_pmdec_corr1,dec_pmdec_corr2,parallax_pmra_corr1,parallax_pmra_corr2,parallax_pmdec_corr1,parallax_pmdec_corr2,pmra_pmdec_corr1,pmra_pmdec_corr2,astrometric_n_obs_al1,astrometric_n_obs_al2,astrometric_n_obs_ac1,astrometric_n_obs_ac2,astrometric_n_good_obs_al1,astrometric_n_good_obs_al2,astrometric_n_bad_obs_al1,astrometric_n_bad_obs_al2,astrometric_gof_al1,astrometric_gof_al2,astrometric_chi2_al1,astrometric_chi2_al2,astrometric_excess_noise1,astrometric_excess_noise2,astrometric_excess_noise_sig1,astrometric_excess_noise_sig2,astrometric_params_solved1,astrometric_params_solved2,astrometric_primary_flag1,astrometric_primary_flag2,nu_eff_used_in_astrometry1,nu_eff_used_in_astrometry2,pseudocolour1,pseudocolour2,pseudocolour_error1,pseudocolour_error2,ra_pseudocolour_corr1,ra_pseudocolour_corr2,dec_pseudocolour_corr1,dec_pseudocolour_corr2,parallax_pseudocolour_corr1,parallax_pseudocolour_corr2,pmra_pseudocolour_corr1,pmra_pseudocolour_corr2,pmdec_pseudocolour_corr1,pmdec_pseudocolour_corr2,astrometric_matched_transits1,astrometric_matched_transits2,visibility_periods_used1,visibility_periods_used2,astrometric_sigma5d_max1,astrometric_sigma5d_max2,matched_transits1,matched_transits2,new_matched_transits1,new_matched_transits2,matched_transits_removed1,matched_transits_removed2,ipd_gof_harmonic_amplitude1,ipd_gof_harmonic_amplitude2,ipd_gof_harmonic_phase1,ipd_gof_harmonic_phase2,ipd_frac_multi_peak1,ipd_frac_multi_peak2,ipd_frac_odd_win1,ipd_frac_odd_win2,ruwe1,ruwe2,scan_direction_strength_k11,scan_direction_strength_k12,scan_direction_strength_k21,scan_direction_strength_k22,scan_direction_strength_k31,scan_direction_strength_k32,scan_direction_strength_k41,scan_direction_strength_k42,scan_direction_mean_k11,scan_direction_mean_k12,scan_direction_mean_k21,scan_direction_mean_k22,scan_direction_mean_k31,scan_direction_mean_k32,scan_direction_mean_k41,scan_direction_mean_k42,duplicated_source1,duplicated_source2,phot_g_n_obs1,phot_g_n_obs2,phot_g_mean_flux1,phot_g_mean_flux2,phot_g_mean_flux_error1,phot_g_mean_flux_error2,phot_g_mean_flux_over_error1,phot_g_mean_flux_over_error2,phot_g_mean_mag1,phot_g_mean_mag2,phot_bp_n_obs1,phot_bp_n_obs2,phot_bp_mean_flux1,phot_bp_mean_flux2,phot_bp_mean_flux_error1,phot_bp_mean_flux_error2,phot_bp_mean_flux_over_error1,phot_bp_mean_flux_over_error2,phot_bp_mean_mag1,phot_bp_mean_mag2,phot_rp_n_obs1,phot_rp_n_obs2,phot_rp_mean_flux1,phot_rp_mean_flux2,phot_rp_mean_flux_error1,phot_rp_mean_flux_error2,phot_rp_mean_flux_over_error1,phot_rp_mean_flux_over_error2,phot_rp_mean_mag1,phot_rp_mean_mag2,phot_bp_n_contaminated_transits1,phot_bp_n_contaminated_transits2,phot_bp_n_blended_transits1,phot_bp_n_blended_transits2,phot_rp_n_contaminated_transits1,phot_rp_n_contaminated_transits2,phot_rp_n_blended_transits1,phot_rp_n_blended_transits2,phot_proc_mode1,phot_proc_mode2,phot_bp_rp_excess_factor1,phot_bp_rp_excess_factor2,bp_rp1,bp_rp2,bp_g1,bp_g2,g_rp1,g_rp2,dr2_radial_velocity1,dr2_radial_velocity2,dr2_radial_velocity_error1,dr2_radial_velocity_error2,dr2_rv_nb_transits1,dr2_rv_nb_transits2,dr2_rv_template_teff1,dr2_rv_template_teff2,dr2_rv_template_logg1,dr2_rv_template_logg2,dr2_rv_template_fe_h1,dr2_rv_template_fe_h2,l1,l2,b1,b2,ecl_lon1,ecl_lon2,ecl_lat1,ecl_lat2,pairdistance,sep_AU,binary_type,Sigma18,R_chance_align,dr2_source_id1,dr2_source_id2,dr2_parallax1,dr2_parallax2,dr2_parallax_error1,dr2_parallax_error2,dr2_pmra1,dr2_pmra2,dr2_pmdec1,dr2_pmdec2,dr2_pmra_

In [23]:
print(ms_sdss5_wd_sdss4[ms_sdss5_wd_sdss4['ms_source_id'] == 875615158730751488])

ms_pos_id solution_id1 solution_id2 ... uses_sdss5 wd_rv_from ms_rv_from
                                    ...                                 
--------- ------------ ------------ ... ---------- ---------- ----------
